# Test Predictions Generator

## Environment Set Up

### Imports

In [1]:
# Data / MONAI
import pandas as pd

# Utils
from Inference import calculate_metrics

### Config

In [2]:
seed = 33
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)

## Load Train Data

In [3]:
subject_ids = pd.read_csv('../data/TRAIN.csv')['SubjectID'].values

ah_segs, unet_segs, segresnet_segs, untr_segs, gt_segs = [], [], [], [], []
for sid in subject_ids:
    ah_channels, unet_channels, segresnet_channels, untr_channels, gt_channels = [], [], [], [], []
    for channel in ['TC', 'WT', 'ET']:
        ah_channels.append(f'../outputs/AHNet/pred_segs/train_pred_segs/pred_{sid}_{channel}.npz')
        unet_channels.append(f'../outputs/UNet/pred_segs/train_pred_segs/pred_{sid}_{channel}.npz')
        segresnet_channels.append(f'../outputs/SegResNet/pred_segs/train_pred_segs/pred_{sid}_{channel}.npz')
        untr_channels.append(f'../outputs/UNETR/pred_segs/train_pred_segs/pred_{sid}_{channel}.npz')
        gt_channels.append(f'../outputs/gt_segs/train_gt_segs/gt_{sid}_{channel}.npz')
    
    ah_segs.append(ah_channels)
    unet_segs.append(unet_channels)
    segresnet_segs.append(segresnet_channels)
    untr_segs.append(untr_channels)
    gt_segs.append(gt_channels)

# Dataframe
train_df = pd.DataFrame()
train_df['SubjectID'] = subject_ids
train_df['AHNet'] = ah_segs
train_df['UNet'] = unet_segs
train_df['SegResNet'] = segresnet_segs
train_df['UNETR'] = untr_segs
train_df['GT'] = gt_segs

train_df.head()

,SubjectID,AHNet,UNet,SegResNet,UNETR,GT
0,100381A,"[../outputs/AHNet/pred_segs/train_pred_segs/pred_100381A_TC.npz, ../outputs/AHNet/pred_segs/train_pred_segs/pred_100381A_WT.npz, ../outputs/AHNet/pred_segs/train_pred_segs/pred_100381A_ET.npz]","[../outputs/UNet/pred_segs/train_pred_segs/pred_100381A_TC.npz, ../outputs/UNet/pred_segs/train_pred_segs/pred_100381A_WT.npz, ../outputs/UNet/pred_segs/train_pred_segs/pred_100381A_ET.npz]","[../outputs/SegResNet/pred_segs/train_pred_segs/pred_100381A_TC.npz, ../outputs/SegResNet/pred_segs/train_pred_segs/pred_100381A_WT.npz, ../outputs/SegResNet/pred_segs/train_pred_segs/pred_100381A_ET.npz]","[../outputs/UNETR/pred_segs/train_pred_segs/pred_100381A_TC.npz, ../outputs/UNETR/pred_segs/train_pred_segs/pred_100381A_WT.npz, ../outputs/UNETR/pred_segs/train_pred_segs/pred_100381A_ET.npz]","[../outputs/gt_segs/train_gt_segs/gt_100381A_TC.npz, ../outputs/gt_segs/train_gt_segs/gt_100381A_WT.npz, ../outputs/gt_segs/train_gt_segs/gt_100381A_ET.npz]"
1,100414B,"[../outputs/AHNet/pred_segs/train_pred_segs/pred_100414B_TC.npz, ../outputs/AHNet/pred_segs/train_pred_segs/pred_100414B_WT.npz, ../outputs/AHNet/pred_segs/train_pred_segs/pred_100414B_ET.npz]","[../outputs/UNet/pred_segs/train_pred_segs/pred_100414B_TC.npz, ../outputs/UNet/pred_segs/train_pred_segs/pred_100414B_WT.npz, ../outputs/UNet/pred_segs/train_pred_segs/pred_100414B_ET.npz]","[../outputs/SegResNet/pred_segs/train_pred_segs/pred_100414B_TC.npz, ../outputs/SegResNet/pred_segs/train_pred_segs/pred_100414B_WT.npz, ../outputs/SegResNet/pred_segs/train_pred_segs/pred_100414B_ET.npz]","[../outputs/UNETR/pred_segs/train_pred_segs/pred_100414B_TC.npz, ../outputs/UNETR/pred_segs/train_pred_segs/pred_100414B_WT.npz, ../outputs/UNETR/pred_segs/train_pred_segs/pred_100414B_ET.npz]","[../outputs/gt_segs/train_gt_segs/gt_100414B_TC.npz, ../outputs/gt_segs/train_gt_segs/gt_100414B_WT.npz, ../outputs/gt_segs/train_gt_segs/gt_100414B_ET.npz]"
2,100132B,"[../outputs/AHNet/pred_segs/train_pred_segs/pred_100132B_TC.npz, ../outputs/AHNet/pred_segs/train_pred_segs/pred_100132B_WT.npz, ../outputs/AHNet/pred_segs/train_pred_segs/pred_100132B_ET.npz]","[../outputs/UNet/pred_segs/train_pred_segs/pred_100132B_TC.npz, ../outputs/UNet/pred_segs/train_pred_segs/pred_100132B_WT.npz, ../outputs/UNet/pred_segs/train_pred_segs/pred_100132B_ET.npz]","[../outputs/SegResNet/pred_segs/train_pred_segs/pred_100132B_TC.npz, ../outputs/SegResNet/pred_segs/train_pred_segs/pred_100132B_WT.npz, ../outputs/SegResNet/pred_segs/train_pred_segs/pred_100132B_ET.npz]","[../outputs/UNETR/pred_segs/train_pred_segs/pred_100132B_TC.npz, ../outputs/UNETR/pred_segs/train_pred_segs/pred_100132B_WT.npz, ../outputs/UNETR/pred_segs/train_pred_segs/pred_100132B_ET.npz]","[../outputs/gt_segs/train_gt_segs/gt_100132B_TC.npz, ../outputs/gt_segs/train_gt_segs/gt_100132B_WT.npz, ../outputs/gt_segs/train_gt_segs/gt_100132B_ET.npz]"
3,100212A,"[../outputs/AHNet/pred_segs/train_pred_segs/pred_100212A_TC.npz, ../outputs/AHNet/pred_segs/train_pred_segs/pred_100212A_WT.npz, ../outputs/AHNet/pred_segs/train_pred_segs/pred_100212A_ET.npz]","[../outputs/UNet/pred_segs/train_pred_segs/pred_100212A_TC.npz, ../outputs/UNet/pred_segs/train_pred_segs/pred_100212A_WT.npz, ../outputs/UNet/pred_segs/train_pred_segs/pred_100212A_ET.npz]","[../outputs/SegResNet/pred_segs/train_pred_segs/pred_100212A_TC.npz, ../outputs/SegResNet/pred_segs/train_pred_segs/pred_100212A_WT.npz, ../outputs/SegResNet/pred_segs/train_pred_segs/pred_100212A_ET.npz]","[../outputs/UNETR/pred_segs/train_pred_segs/pred_100212A_TC.npz, ../outputs/UNETR/pred_segs/train_pred_segs/pred_100212A_WT.npz, ../outputs/UNETR/pred_segs/train_pred_segs/pred_100212A_ET.npz]","[../outputs/gt_segs/train_gt_segs/gt_100212A_TC.npz, ../outputs/gt_segs/train_gt_segs/gt_100212A_WT.npz, ../outputs/gt_segs/train_gt_segs/gt_100212A_ET.npz]"
4,100243B,"[../outputs/AHNet/pred_segs/train_pred_segs/pred_100243B_TC.npz, ../outputs/AHNet/pred_segs/train_pred_segs/pred_100

## Load Val Data

In [4]:
subject_ids = pd.read_csv('../data/VAL.csv')['SubjectID'].values

ah_segs, unet_segs, segresnet_segs, untr_segs, gt_segs = [], [], [], [], []
for sid in subject_ids:
    ah_channels, unet_channels, segresnet_channels, untr_channels, gt_channels = [], [], [], [], []
    for channel in ['TC', 'WT', 'ET']:
        ah_channels.append(f'../outputs/AHNet/pred_segs/val_pred_segs/pred_{sid}_{channel}.npz')
        unet_channels.append(f'../outputs/UNet/pred_segs/val_pred_segs/pred_{sid}_{channel}.npz')
        segresnet_channels.append(f'../outputs/SegResNet/pred_segs/val_pred_segs/pred_{sid}_{channel}.npz')
        untr_channels.append(f'../outputs/UNETR/pred_segs/val_pred_segs/pred_{sid}_{channel}.npz')
        gt_channels.append(f'../outputs/gt_segs/val_gt_segs/gt_{sid}_{channel}.npz')
    
    ah_segs.append(ah_channels)
    unet_segs.append(unet_channels)
    segresnet_segs.append(segresnet_channels)
    untr_segs.append(untr_channels)
    gt_segs.append(gt_channels)

# Dataframe
val_df = pd.DataFrame()
val_df['SubjectID'] = subject_ids
val_df['AHNet'] = ah_segs
val_df['UNet'] = unet_segs
val_df['SegResNet'] = segresnet_segs
val_df['UNETR'] = untr_segs
val_df['GT'] = gt_segs

val_df.head()

,SubjectID,AHNet,UNet,SegResNet,UNETR,GT
0,100237A,"[../outputs/AHNet/pred_segs/val_pred_segs/pred_100237A_TC.npz, ../outputs/AHNet/pred_segs/val_pred_segs/pred_100237A_WT.npz, ../outputs/AHNet/pred_segs/val_pred_segs/pred_100237A_ET.npz]","[../outputs/UNet/pred_segs/val_pred_segs/pred_100237A_TC.npz, ../outputs/UNet/pred_segs/val_pred_segs/pred_100237A_WT.npz, ../outputs/UNet/pred_segs/val_pred_segs/pred_100237A_ET.npz]","[../outputs/SegResNet/pred_segs/val_pred_segs/pred_100237A_TC.npz, ../outputs/SegResNet/pred_segs/val_pred_segs/pred_100237A_WT.npz, ../outputs/SegResNet/pred_segs/val_pred_segs/pred_100237A_ET.npz]","[../outputs/UNETR/pred_segs/val_pred_segs/pred_100237A_TC.npz, ../outputs/UNETR/pred_segs/val_pred_segs/pred_100237A_WT.npz, ../outputs/UNETR/pred_segs/val_pred_segs/pred_100237A_ET.npz]","[../outputs/gt_segs/val_gt_segs/gt_100237A_TC.npz, ../outputs/gt_segs/val_gt_segs/gt_100237A_WT.npz, ../outputs/gt_segs/val_gt_segs/gt_100237A_ET.npz]"
1,100219A,"[../outputs/AHNet/pred_segs/val_pred_segs/pred_100219A_TC.npz, ../outputs/AHNet/pred_segs/val_pred_segs/pred_100219A_WT.npz, ../outputs/AHNet/pred_segs/val_pred_segs/pred_100219A_ET.npz]","[../outputs/UNet/pred_segs/val_pred_segs/pred_100219A_TC.npz, ../outputs/UNet/pred_segs/val_pred_segs/pred_100219A_WT.npz, ../outputs/UNet/pred_segs/val_pred_segs/pred_100219A_ET.npz]","[../outputs/SegResNet/pred_segs/val_pred_segs/pred_100219A_TC.npz, ../outputs/SegResNet/pred_segs/val_pred_segs/pred_100219A_WT.npz, ../outputs/SegResNet/pred_segs/val_pred_segs/pred_100219A_ET.npz]","[../outputs/UNETR/pred_segs/val_pred_segs/pred_100219A_TC.npz, ../outputs/UNETR/pred_segs/val_pred_segs/pred_100219A_WT.npz, ../outputs/UNETR/pred_segs/val_pred_segs/pred_100219A_ET.npz]","[../outputs/gt_segs/val_gt_segs/gt_100219A_TC.npz, ../outputs/gt_segs/val_gt_segs/gt_100219A_WT.npz, ../outputs/gt_segs/val_gt_segs/gt_100219A_ET.npz]"
2,100363A,"[../outputs/AHNet/pred_segs/val_pred_segs/pred_100363A_TC.npz, ../outputs/AHNet/pred_segs/val_pred_segs/pred_100363A_WT.npz, ../outputs/AHNet/pred_segs/val_pred_segs/pred_100363A_ET.npz]","[../outputs/UNet/pred_segs/val_pred_segs/pred_100363A_TC.npz, ../outputs/UNet/pred_segs/val_pred_segs/pred_100363A_WT.npz, ../outputs/UNet/pred_segs/val_pred_segs/pred_100363A_ET.npz]","[../outputs/SegResNet/pred_segs/val_pred_segs/pred_100363A_TC.npz, ../outputs/SegResNet/pred_segs/val_pred_segs/pred_100363A_WT.npz, ../outputs/SegResNet/pred_segs/val_pred_segs/pred_100363A_ET.npz]","[../outputs/UNETR/pred_segs/val_pred_segs/pred_100363A_TC.npz, ../outputs/UNETR/pred_segs/val_pred_segs/pred_100363A_WT.npz, ../outputs/UNETR/pred_segs/val_pred_segs/pred_100363A_ET.npz]","[../outputs/gt_segs/val_gt_segs/gt_100363A_TC.npz, ../outputs/gt_segs/val_gt_segs/gt_100363A_WT.npz, ../outputs/gt_segs/val_gt_segs/gt_100363A_ET.npz]"
3,100354A,"[../outputs/AHNet/pred_segs/val_pred_segs/pred_100354A_TC.npz, ../outputs/AHNet/pred_segs/val_pred_segs/pred_100354A_WT.npz, ../outputs/AHNet/pred_segs/val_pred_segs/pred_100354A_ET.npz]","[../outputs/UNet/pred_segs/val_pred_segs/pred_100354A_TC.npz, ../outputs/UNet/pred_segs/val_pred_segs/pred_100354A_WT.npz, ../outputs/UNet/pred_segs/val_pred_segs/pred_100354A_ET.npz]","[../outputs/SegResNet/pred_segs/val_pred_segs/pred_100354A_TC.npz, ../outputs/SegResNet/pred_segs/val_pred_segs/pred_100354A_WT.npz, ../outputs/SegResNet/pred_segs/val_pred_segs/pred_100354A_ET.npz]","[../outputs/UNETR/pred_segs/val_pred_segs/pred_100354A_TC.npz, ../outputs/UNETR/pred_segs/val_pred_segs/pred_100354A_WT.npz, ../outputs/UNETR/pred_segs/val_pred_segs/pred_100354A_ET.npz]","[../outputs/gt_segs/val_gt_segs/gt_100354A_TC.npz, ../outputs/gt_segs/val_gt_segs/gt_100354A_WT.npz, ../outputs/gt_segs/val_gt_segs/gt_100354A_ET.npz]"
4,100303A,"[../outputs/AHNet/pred_segs/val_pred_segs/pred_100303A_TC.npz, ../outputs/AHNet/pred_segs/val_pred_segs/pred_100303A_WT.npz, ../outputs/AHNet/pred_segs/val_pred_segs/pred_100303A_ET.npz]","[../outputs/UNet/pred_segs/val_pred_segs/pred_1

## Load Test Data

In [5]:
subject_ids = pd.read_csv('../data/TEST.csv')['SubjectID'].values

ah_segs, unet_segs, segresnet_segs, untr_segs, gt_segs = [], [], [], [], []
for sid in subject_ids:
    ah_channels, unet_channels, segresnet_channels, untr_channels, gt_channels = [], [], [], [], []
    for channel in ['TC', 'WT', 'ET']:
        ah_channels.append(f'../outputs/AHNet/pred_segs/test_pred_segs/pred_{sid}_{channel}.npz')
        unet_channels.append(f'../outputs/UNet/pred_segs/test_pred_segs/pred_{sid}_{channel}.npz')
        segresnet_channels.append(f'../outputs/SegResNet/pred_segs/test_pred_segs/pred_{sid}_{channel}.npz')
        untr_channels.append(f'../outputs/UNETR/pred_segs/test_pred_segs/pred_{sid}_{channel}.npz')
        gt_channels.append(f'../outputs/gt_segs/test_gt_segs/gt_{sid}_{channel}.npz')
    
    ah_segs.append(ah_channels)
    unet_segs.append(unet_channels)
    segresnet_segs.append(segresnet_channels)
    untr_segs.append(untr_channels)
    gt_segs.append(gt_channels)

# Dataframe
test_df = pd.DataFrame()
test_df['SubjectID'] = subject_ids
test_df['AHNet'] = ah_segs
test_df['UNet'] = unet_segs
test_df['SegResNet'] = segresnet_segs
test_df['UNETR'] = untr_segs
test_df['GT'] = gt_segs

test_df.head()

,SubjectID,AHNet,UNet,SegResNet,UNETR,GT
0,100214B,"[../outputs/AHNet/pred_segs/test_pred_segs/pred_100214B_TC.npz, ../outputs/AHNet/pred_segs/test_pred_segs/pred_100214B_WT.npz, ../outputs/AHNet/pred_segs/test_pred_segs/pred_100214B_ET.npz]","[../outputs/UNet/pred_segs/test_pred_segs/pred_100214B_TC.npz, ../outputs/UNet/pred_segs/test_pred_segs/pred_100214B_WT.npz, ../outputs/UNet/pred_segs/test_pred_segs/pred_100214B_ET.npz]","[../outputs/SegResNet/pred_segs/test_pred_segs/pred_100214B_TC.npz, ../outputs/SegResNet/pred_segs/test_pred_segs/pred_100214B_WT.npz, ../outputs/SegResNet/pred_segs/test_pred_segs/pred_100214B_ET.npz]","[../outputs/UNETR/pred_segs/test_pred_segs/pred_100214B_TC.npz, ../outputs/UNETR/pred_segs/test_pred_segs/pred_100214B_WT.npz, ../outputs/UNETR/pred_segs/test_pred_segs/pred_100214B_ET.npz]","[../outputs/gt_segs/test_gt_segs/gt_100214B_TC.npz, ../outputs/gt_segs/test_gt_segs/gt_100214B_WT.npz, ../outputs/gt_segs/test_gt_segs/gt_100214B_ET.npz]"
1,100340A,"[../outputs/AHNet/pred_segs/test_pred_segs/pred_100340A_TC.npz, ../outputs/AHNet/pred_segs/test_pred_segs/pred_100340A_WT.npz, ../outputs/AHNet/pred_segs/test_pred_segs/pred_100340A_ET.npz]","[../outputs/UNet/pred_segs/test_pred_segs/pred_100340A_TC.npz, ../outputs/UNet/pred_segs/test_pred_segs/pred_100340A_WT.npz, ../outputs/UNet/pred_segs/test_pred_segs/pred_100340A_ET.npz]","[../outputs/SegResNet/pred_segs/test_pred_segs/pred_100340A_TC.npz, ../outputs/SegResNet/pred_segs/test_pred_segs/pred_100340A_WT.npz, ../outputs/SegResNet/pred_segs/test_pred_segs/pred_100340A_ET.npz]","[../outputs/UNETR/pred_segs/test_pred_segs/pred_100340A_TC.npz, ../outputs/UNETR/pred_segs/test_pred_segs/pred_100340A_WT.npz, ../outputs/UNETR/pred_segs/test_pred_segs/pred_100340A_ET.npz]","[../outputs/gt_segs/test_gt_segs/gt_100340A_TC.npz, ../outputs/gt_segs/test_gt_segs/gt_100340A_WT.npz, ../outputs/gt_segs/test_gt_segs/gt_100340A_ET.npz]"
2,100391A,"[../outputs/AHNet/pred_segs/test_pred_segs/pred_100391A_TC.npz, ../outputs/AHNet/pred_segs/test_pred_segs/pred_100391A_WT.npz, ../outputs/AHNet/pred_segs/test_pred_segs/pred_100391A_ET.npz]","[../outputs/UNet/pred_segs/test_pred_segs/pred_100391A_TC.npz, ../outputs/UNet/pred_segs/test_pred_segs/pred_100391A_WT.npz, ../outputs/UNet/pred_segs/test_pred_segs/pred_100391A_ET.npz]","[../outputs/SegResNet/pred_segs/test_pred_segs/pred_100391A_TC.npz, ../outputs/SegResNet/pred_segs/test_pred_segs/pred_100391A_WT.npz, ../outputs/SegResNet/pred_segs/test_pred_segs/pred_100391A_ET.npz]","[../outputs/UNETR/pred_segs/test_pred_segs/pred_100391A_TC.npz, ../outputs/UNETR/pred_segs/test_pred_segs/pred_100391A_WT.npz, ../outputs/UNETR/pred_segs/test_pred_segs/pred_100391A_ET.npz]","[../outputs/gt_segs/test_gt_segs/gt_100391A_TC.npz, ../outputs/gt_segs/test_gt_segs/gt_100391A_WT.npz, ../outputs/gt_segs/test_gt_segs/gt_100391A_ET.npz]"
3,100190B,"[../outputs/AHNet/pred_segs/test_pred_segs/pred_100190B_TC.npz, ../outputs/AHNet/pred_segs/test_pred_segs/pred_100190B_WT.npz, ../outputs/AHNet/pred_segs/test_pred_segs/pred_100190B_ET.npz]","[../outputs/UNet/pred_segs/test_pred_segs/pred_100190B_TC.npz, ../outputs/UNet/pred_segs/test_pred_segs/pred_100190B_WT.npz, ../outputs/UNet/pred_segs/test_pred_segs/pred_100190B_ET.npz]","[../outputs/SegResNet/pred_segs/test_pred_segs/pred_100190B_TC.npz, ../outputs/SegResNet/pred_segs/test_pred_segs/pred_100190B_WT.npz, ../outputs/SegResNet/pred_segs/test_pred_segs/pred_100190B_ET.npz]","[../outputs/UNETR/pred_segs/test_pred_segs/pred_100190B_TC.npz, ../outputs/UNETR/pred_segs/test_pred_segs/pred_100190B_WT.npz, ../outputs/UNETR/pred_segs/test_pred_segs/pred_100190B_ET.npz]","[../outputs/gt_segs/test_gt_segs/gt_100190B_TC.npz, ../outputs/gt_segs/test_gt_segs/gt_100190B_WT.npz, ../outputs/gt_segs/test_gt_segs/gt_100190B_ET.npz]"
4,100142A,"[../outputs/AHNet/pred_segs/test_pred_segs/pred_100142A_TC.npz, ../outputs/AHNet/pred_segs/test_pred_segs/pred_100142A_WT.npz, ../outputs/AHNet/pred_segs/test_pred_segs/pred_10

## Generate Results CSV

### AHNet

In [6]:
# VAL AHnet
best_mean = 0
best_threshold = 0
df = None
for threshold in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    # VAL AHNet
    ahnet_df = calculate_metrics('AHNet', val_df, threshold)
    mean = ahnet_df['Dice'].mean()
    print(f'AHNet VAL Mean Dice for threshold {threshold}: {mean}')
    if mean > best_mean:
        best_mean = mean
        best_threshold = threshold
        df = ahnet_df
print(f'BEST THRESHOLD: {best_threshold}')
df.to_csv('../outputs/AHNet/val_scores.csv', index=False)
df.describe()

AHNet VAL Mean Dice for threshold 0.1: 0.5492611799028612
AHNet VAL Mean Dice for threshold 0.2: 0.5604259324890952
AHNet VAL Mean Dice for threshold 0.3: 0.5605223681627503
AHNet VAL Mean Dice for threshold 0.4: 0.5556399243162765
AHNet VAL Mean Dice for threshold 0.5: 0.5462690406899539
AHNet VAL Mean Dice for threshold 0.6: 0.5356928398532252
AHNet VAL Mean Dice for threshold 0.7: 0.5207302911627677
AHNet VAL Mean Dice for threshold 0.8: 0.49963182919929106
AHNet VAL Mean Dice for threshold 0.9: 0.46508253750301176
BEST THRESHOLD: 0.3


,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.560522,0.595008,0.599129,0.481108,12.774194,10.258065,109.290323,7.741935,7.741935,3.032258,21333.903226,21511.290323,19288.032258,19105.225806,19355.193548,15780.967742
std,0.291371,0.294258,0.299808,0.303951,12.416950,10.318164,241.633220,10.708152,10.708152,2.469600,41954.305231,42270.831368,38737.120564,37451.165077,37730.849620,32728.506884
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,69.000000,69.000000,0.000000
25%,0.333367,0.361916,0.361883,0.273857,5.000000,4.000000,6.000000,2.500000,2.500000,1.500000,597.500000,599.000000,416.500000,606.500000,606.500000,271.000000
50%,0.639329,0.709020,0.712976,0.477851,8.000000,7.000000,19.000000,5.000000,5.000000,2.000000,1542.000000,1532.000000,1161.000000,2111.000000,2149.000000,949.000000
75%,0.837559,0.855369,0.865494,0.783561,18.000000,14.000000,61.000000,8.000000,8.000000,4.000000,21160.500000,21425.000000,18357.500000,20289.000000,20793.500000,16180.500000
max,0.928312,0.933032,0.939695,0.912208,57.000000,42.000000,1095.000000,55.000000,55.000000,9.000000,171800.000000,173245.000000,158599.000000,153858.000000,153901.000000,133325.000000


In [7]:
# TRAIN AHNet
ahnet_df = calculate_metrics('AHNet', train_df, best_threshold)
ahnet_df.to_csv('../outputs/AHNet/train_scores.csv', index=False)
ahnet_df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000
mean,0.723566,0.756423,0.765443,0.625716,14.040323,10.580645,91.544355,8.858871,8.854839,5.112903,23672.471774,23899.387097,21499.774194,21093.334677,21733.947581,17351.237903
std,0.184608,0.174877,0.175895,0.256993,12.070910,8.853826,169.174149,9.310666,9.303862,5.076276,34480.329453,34788.361113,31820.207362,31059.680139,31737.526216,27439.413513
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,14.000000,14.000000,0.000000
25%,0.646455,0.702860,0.720909,0.492891,5.000000,4.000000,8.000000,3.000000,3.000000,1.750000,1445.750000,1478.750000,1128.250000,1370.000000,1387.250000,685.750000
50%,0.777697,0.809578,0.820805,0.715839,11.000000,8.000000,24.000000,5.000000,5.000000,3.000000,9122.500000,9195.500000,7680.500000,7986.000000,8314.000000,5959.000000
75%,0.856928,0.880109,0.884072,0.821554,18.250000,15.000000,92.250000,12.000000,12.000000,7.000000,32321.000000,32614.500000,29109.500000,28620.500000,28902.000000,23018.250000
max,0.932762,0.937002,0.940889,0.925134,77.000000,62.000000,1176.000000,48.000000,48.000000,36.000000,198358.000000,199488.000000,180157.000000,181402.000000,183395.000000,162650.000000


In [8]:
# TEST AHNet
ahnet_df = calculate_metrics('AHNet', test_df, best_threshold)
ahnet_df.to_csv('../outputs/AHNet/test_scores.csv', index=False)
ahnet_df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.583364,0.617662,0.626915,0.501632,10.580645,7.870968,69.064516,8.709677,8.709677,5.129032,18751.741935,18937.580645,17134.516129,17105.387097,17985.870968,13810.806452
std,0.273949,0.272766,0.278902,0.288378,8.135413,6.652027,130.761089,9.103822,9.103822,4.951376,28004.605846,28278.026189,26452.299711,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,5.000000,0.000000,0.000000,44.000000,44.000000,0.000000
25%,0.417180,0.489340,0.488268,0.272500,5.500000,3.000000,7.000000,2.000000,2.000000,1.500000,911.500000,914.500000,815.500000,802.000000,802.000000,410.000000
50%,0.648561,0.659333,0.694444,0.561328,7.000000,7.000000,16.000000,6.000000,6.000000,4.000000,4825.000000,4866.000000,3967.000000,3847.000000,4138.000000,1944.000000
75%,0.821729,0.853907,0.872481,0.732228,15.000000,10.000000,60.500000,13.500000,13.500000,7.000000,33080.500000,33513.500000,32337.500000,26305.000000,27036.500000,23520.500000
max,0.920359,0.934362,0.933894,0.892820,34.000000,26.000000,588.000000,43.000000,43.000000,21.000000,107917.000000,108916.000000,101786.000000,102770.000000,105729.000000,92221.000000


### SegResNet

In [9]:
# VAL SegResNet
best_mean = 0
best_threshold = 0
df = None
for threshold in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    # SegResNet
    segresnet_df = calculate_metrics('SegResNet', val_df, threshold)
    mean = segresnet_df['Dice'].mean()
    print(f'SegResNet VAL Mean Dice for threshold {threshold}: {mean}')
    if mean > best_mean:
        best_mean = mean
        best_threshold = threshold
        df = segresnet_df
print(f'BEST THRESHOLD: {best_threshold}')
df.to_csv('../outputs/SegResNet/val_scores.csv', index=False)
df.describe()

SegResNet VAL Mean Dice for threshold 0.1: 0.5970835380196091
SegResNet VAL Mean Dice for threshold 0.2: 0.6291356047074641
SegResNet VAL Mean Dice for threshold 0.3: 0.642114634505443
SegResNet VAL Mean Dice for threshold 0.4: 0.646779386567012
SegResNet VAL Mean Dice for threshold 0.5: 0.6486533776887001
SegResNet VAL Mean Dice for threshold 0.6: 0.6453569404059841
SegResNet VAL Mean Dice for threshold 0.7: 0.635466123660726
SegResNet VAL Mean Dice for threshold 0.8: 0.6187912326065763
SegResNet VAL Mean Dice for threshold 0.9: 0.5839838963903247
BEST THRESHOLD: 0.5


,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.00000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.648653,0.682256,0.689264,0.554394,7.129032,6.83871,4.129032,7.741935,7.741935,3.032258,18359.354839,18367.225806,15318.516129,19105.225806,19355.193548,15780.967742
std,0.237255,0.229664,0.233772,0.296713,5.439007,4.89283,3.344069,10.708152,10.708152,2.469600,34927.167462,35135.087426,30755.426912,37451.165077,37730.849620,32728.506884
min,0.017208,0.017117,0.017299,0.000000,1.000000,1.00000,1.000000,1.000000,1.000000,0.000000,135.000000,126.000000,13.000000,69.000000,69.000000,0.000000
25%,0.474633,0.535691,0.525300,0.373331,4.000000,4.00000,2.000000,2.500000,2.500000,1.500000,659.000000,682.500000,243.500000,606.500000,606.500000,271.000000
50%,0.734192,0.772202,0.783077,0.571030,6.000000,5.00000,3.000000,5.000000,5.000000,2.000000,1190.000000,1167.000000,759.000000,2111.000000,2149.000000,949.000000
75%,0.835967,0.848231,0.875156,0.817324,9.000000,8.50000,5.500000,8.000000,8.000000,4.000000,16917.000000,16844.000000,13720.000000,20289.000000,20793.500000,16180.500000
max,0.949027,0.946284,0.955822,0.944976,29.000000,25.00000,17.000000,55.000000,55.000000,9.000000,135459.000000,136617.000000,121015.000000,153858.000000,153901.000000,133325.000000


In [10]:
# TRAIN SegResNet
segresnet_df = calculate_metrics('SegResNet', train_df, best_threshold)
segresnet_df.to_csv('../outputs/SegResNet/train_scores.csv', index=False)
segresnet_df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000
mean,0.768484,0.785704,0.796827,0.698076,7.669355,7.512097,5.439516,8.858871,8.854839,5.112903,20147.689516,20166.741935,16357.927419,21093.334677,21733.947581,17351.237903
std,0.164607,0.156092,0.154724,0.247429,5.479031,5.442360,4.564026,9.310666,9.303862,5.076276,29012.586162,29184.759260,25194.591465,31059.680139,31737.526216,27439.413513
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,31.000000,38.000000,0.000000,14.000000,14.000000,0.000000
25%,0.700824,0.723394,0.743175,0.591442,4.000000,4.000000,2.000000,3.000000,3.000000,1.750000,1339.000000,1327.250000,695.750000,1370.000000,1387.250000,685.750000
50%,0.808852,0.825847,0.847520,0.778233,6.000000,6.000000,4.000000,5.000000,5.000000,3.000000,8043.000000,7995.000000,5308.000000,7986.000000,8314.000000,5959.000000
75%,0.898070,0.906055,0.913108,0.880314,10.000000,10.000000,7.000000,12.000000,12.000000,7.000000,27568.000000,27518.750000,21589.750000,28620.500000,28902.000000,23018.250000
max,0.957516,0.960225,0.961582,0.957261,31.000000,31.000000,29.000000,48.000000,48.000000,36.000000,162702.000000,164809.000000,144557.000000,181402.000000,183395.000000,162650.000000


In [11]:
# TEST SegResNet
segresnet_df = calculate_metrics('SegResNet', test_df, best_threshold)
segresnet_df.to_csv('../outputs/SegResNet/test_scores.csv', index=False)
segresnet_df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.646272,0.679585,0.684750,0.561635,8.451613,8.354839,5.451613,8.709677,8.709677,5.129032,14533.483871,14518.193548,11519.967742,17105.387097,17985.870968,13810.806452
std,0.237641,0.217268,0.228583,0.294801,6.850006,6.848593,5.271551,9.103822,9.103822,4.951376,21299.762596,21376.291349,18354.865167,25565.209783,26912.616906,22937.645993
min,0.084220,0.151394,0.101266,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,71.000000,63.000000,0.000000,44.000000,44.000000,0.000000
25%,0.484024,0.572342,0.567735,0.324266,4.500000,3.500000,1.000000,2.000000,2.000000,1.500000,1100.000000,1081.000000,636.000000,802.000000,802.000000,410.000000
50%,0.723051,0.763834,0.768185,0.638653,6.000000,6.000000,4.000000,6.000000,6.000000,4.000000,4401.000000,4441.000000,2781.000000,3847.000000,4138.000000,1944.000000
75%,0.825707,0.855405,0.865770,0.796760,10.500000,10.500000,8.000000,13.500000,13.500000,7.000000,21443.500000,21289.500000,18070.500000,26305.000000,27036.500000,23520.500000
max,0.945202,0.950976,0.952450,0.935746,26.000000,27.000000,26.000000,43.000000,43.000000,21.000000,78216.000000,79071.000000,68497.000000,102770.000000,105729.000000,92221.000000


### UNet

In [12]:
# VAL UNet
best_mean = 0
best_threshold = 0
df = None
for threshold in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    # UNETR
    unet_df = calculate_metrics('UNet', val_df, threshold)
    mean = unet_df['Dice'].mean()
    print(f'UNet VAL Mean Dice for threshold {threshold}: {mean}')
    if mean > best_mean:
        best_mean = mean
        best_threshold = threshold
        df = unet_df
print(f'BEST THRESHOLD: {best_threshold}')
df.to_csv('../outputs/UNet/val_scores.csv', index=False)
df.describe()

UNet VAL Mean Dice for threshold 0.1: 0.4282878165118276
UNet VAL Mean Dice for threshold 0.2: 0.42832267277164293
UNet VAL Mean Dice for threshold 0.3: 0.42831103770708245
UNet VAL Mean Dice for threshold 0.4: 0.4282265670537468
UNet VAL Mean Dice for threshold 0.5: 0.42820052421771954
UNet VAL Mean Dice for threshold 0.6: 0.42809437097410763
UNet VAL Mean Dice for threshold 0.7: 0.42796112505734085
UNet VAL Mean Dice for threshold 0.8: 0.4279389574583019
UNet VAL Mean Dice for threshold 0.9: 0.4277176525642825
BEST THRESHOLD: 0.2


,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.428323,0.453467,0.461658,0.369501,11.419355,10.838710,11.451613,7.741935,7.741935,3.032258,21324.774194,21312.677419,21327.548387,19105.225806,19355.193548,15780.967742
std,0.344629,0.358677,0.366625,0.322365,34.860459,31.846608,34.501052,10.708152,10.708152,2.469600,34065.143266,34056.285313,34061.761377,37451.165077,37730.849620,32728.506884
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1315.000000,1321.000000,1318.000000,69.000000,69.000000,0.000000
25%,0.066284,0.078828,0.077547,0.042478,3.000000,3.000000,3.000000,2.500000,2.500000,1.500000,3118.500000,3116.000000,3121.500000,606.500000,606.500000,271.000000
50%,0.544761,0.582043,0.581235,0.341828,4.000000,4.000000,4.000000,5.000000,5.000000,2.000000,7019.000000,7039.000000,7020.000000,2111.000000,2149.000000,949.000000
75%,0.729432,0.769625,0.832015,0.661509,7.000000,6.500000,7.000000,8.000000,8.000000,4.000000,19793.000000,19788.500000,19789.000000,20289.000000,20793.500000,16180.500000
max,0.913806,0.928784,0.938529,0.874104,198.000000,181.000000,196.000000,55.000000,55.000000,9.000000,139620.000000,139598.000000,139578.000000,153858.000000,153901.000000,133325.000000


In [13]:
# TRAIN UNet
unet_df = calculate_metrics('UNet', train_df, best_threshold)
unet_df.to_csv('../outputs/UNet/train_scores.csv', index=False)
unet_df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000
mean,0.608586,0.639933,0.650746,0.531623,5.161290,4.983871,5.185484,8.858871,8.854839,5.112903,21188.552419,21189.427419,21190.818548,21093.334677,21733.947581,17351.237903
std,0.297599,0.301294,0.304845,0.304939,7.484275,6.112288,7.737983,9.310666,9.303862,5.076276,29412.451566,29405.884187,29412.237876,31059.680139,31737.526216,27439.413513
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1137.000000,1144.000000,1138.000000,14.000000,14.000000,0.000000
25%,0.405228,0.465330,0.468455,0.266761,2.000000,2.000000,2.000000,3.000000,3.000000,1.750000,2941.750000,2939.250000,2939.000000,1370.000000,1387.250000,685.750000
50%,0.719100,0.764661,0.782324,0.620482,4.000000,4.000000,4.000000,5.000000,5.000000,3.000000,8430.500000,8442.500000,8435.500000,7986.000000,8314.000000,5959.000000
75%,0.844796,0.879796,0.885119,0.796361,6.000000,6.000000,6.250000,12.000000,12.000000,7.000000,26556.000000,26550.250000,26555.750000,28620.500000,28902.000000,23018.250000
max,0.941337,0.947561,0.949046,0.927403,107.000000,83.000000,111.000000,48.000000,48.000000,36.000000,177345.000000,177244.000000,177340.000000,181402.000000,183395.000000,162650.000000


In [14]:
# TEST UNet
unet_df = calculate_metrics('UNet', test_df, best_threshold)
unet_df.to_csv('../outputs/UNet/test_scores.csv', index=False)
unet_df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.484883,0.517132,0.529417,0.408100,21.548387,19.161290,23.193548,8.709677,8.709677,5.129032,16574.387097,16540.774194,16578.838710,17105.387097,17985.870968,13810.806452
std,0.328195,0.339524,0.346530,0.307589,62.423200,53.717841,69.292818,9.103822,9.103822,4.951376,20918.777034,20937.924118,20923.296144,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1785.000000,1787.000000,1791.000000,44.000000,44.000000,0.000000
25%,0.116562,0.141904,0.141575,0.086970,3.000000,2.500000,3.000000,2.000000,2.000000,1.500000,3656.500000,3662.500000,3661.500000,802.000000,802.000000,410.000000
50%,0.582786,0.647238,0.647275,0.447944,5.000000,5.000000,5.000000,6.000000,6.000000,4.000000,5500.000000,5513.000000,5516.000000,3847.000000,4138.000000,1944.000000
75%,0.767916,0.834543,0.836938,0.671957,8.500000,8.000000,9.000000,13.500000,13.500000,7.000000,26130.000000,26137.500000,26131.500000,26305.000000,27036.500000,23520.500000
max,0.930020,0.946876,0.949140,0.898466,274.000000,237.000000,328.000000,43.000000,43.000000,21.000000,75747.000000,75778.000000,75750.000000,102770.000000,105729.000000,92221.000000


### UNETR

In [15]:
# VAL UNETR
best_mean = 0
best_threshold = 0
df = None
for threshold in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    # UNETR
    untr_df = calculate_metrics('UNETR', val_df, threshold)
    mean = untr_df['Dice'].mean()
    print(f'UNETR VAL Mean Dice for threshold {threshold}: {mean}')
    if mean > best_mean:
        best_mean = mean
        best_threshold = threshold
        df = untr_df
print(f'BEST THRESHOLD: {best_threshold}')
df.to_csv('../outputs/UNETR/val_scores.csv', index=False)
df.describe()

UNETR VAL Mean Dice for threshold 0.1: 0.5113578857553582
UNETR VAL Mean Dice for threshold 0.2: 0.5365295441701047
UNETR VAL Mean Dice for threshold 0.3: 0.5473637978936876
UNETR VAL Mean Dice for threshold 0.4: 0.5504476521765033
UNETR VAL Mean Dice for threshold 0.5: 0.547998258844018
UNETR VAL Mean Dice for threshold 0.6: 0.5435207280660829
UNETR VAL Mean Dice for threshold 0.7: 0.5328597827183623
UNETR VAL Mean Dice for threshold 0.8: 0.5149041368295589
UNETR VAL Mean Dice for threshold 0.9: 0.4807232413440943
BEST THRESHOLD: 0.4


,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.550448,0.573631,0.581319,0.484506,16.677419,16.967742,10.419355,7.741935,7.741935,3.032258,19717.354839,19936.096774,16438.709677,19105.225806,19355.193548,15780.967742
std,0.300557,0.296484,0.300111,0.335131,20.989501,21.138407,10.626301,10.708152,10.708152,2.469600,36504.672766,37090.636853,32046.198094,37451.165077,37730.849620,32728.506884
min,0.012344,0.012241,0.012447,0.000000,2.000000,2.000000,1.000000,1.000000,1.000000,0.000000,84.000000,96.000000,1.000000,69.000000,69.000000,0.000000
25%,0.308373,0.316527,0.348451,0.171904,7.500000,8.000000,4.500000,2.500000,2.500000,1.500000,744.000000,762.000000,246.000000,606.500000,606.500000,271.000000
50%,0.579267,0.662887,0.680541,0.486154,11.000000,11.000000,7.000000,5.000000,5.000000,2.000000,2739.000000,2738.000000,1661.000000,2111.000000,2149.000000,949.000000
75%,0.796567,0.813837,0.843381,0.775039,15.000000,16.500000,12.000000,8.000000,8.000000,4.000000,18423.000000,18468.000000,13786.000000,20289.000000,20793.500000,16180.500000
max,0.937153,0.932987,0.953532,0.924940,116.000000,117.000000,46.000000,55.000000,55.000000,9.000000,147550.000000,148563.000000,129968.000000,153858.000000,153901.000000,133325.000000


In [16]:
# TRAIN UNETR
untr_df = calculate_metrics('UNETR', train_df, best_threshold)
untr_df.to_csv('../outputs/UNETR/train_scores.csv', index=False)
untr_df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000
mean,0.749103,0.765276,0.775619,0.679929,14.036290,13.899194,10.403226,8.858871,8.854839,5.112903,22036.697581,22138.181452,18321.020161,21093.334677,21733.947581,17351.237903
std,0.182098,0.174623,0.176229,0.256125,12.868798,12.509507,9.776796,9.310666,9.303862,5.076276,30801.743966,30997.352263,26992.466374,31059.680139,31737.526216,27439.413513
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,39.000000,37.000000,0.000000,14.000000,14.000000,0.000000
25%,0.670434,0.710765,0.716475,0.587184,6.000000,6.000000,4.000000,3.000000,3.000000,1.750000,1503.750000,1528.000000,755.000000,1370.000000,1387.250000,685.750000
50%,0.792954,0.808146,0.827765,0.781156,11.000000,11.000000,8.000000,5.000000,5.000000,3.000000,8114.000000,8143.000000,6147.000000,7986.000000,8314.000000,5959.000000
75%,0.881435,0.889505,0.897203,0.868191,17.250000,17.000000,14.000000,12.000000,12.000000,7.000000,30581.750000,30682.750000,24715.250000,28620.500000,28902.000000,23018.250000
max,0.955583,0.955079,0.956506,0.955171,142.000000,135.000000,85.000000,48.000000,48.000000,36.000000,177791.000000,179131.000000,156216.000000,181402.000000,183395.000000,162650.000000


In [17]:
# TEST UNETR
untr_df = calculate_metrics('UNETR', test_df, best_threshold)
untr_df.to_csv('../outputs/UNETR/test_scores.csv', index=False)
untr_df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.545455,0.579272,0.586039,0.466769,16.225806,15.806452,10.258065,8.709677,8.709677,5.129032,15686.032258,15907.258065,13347.096774,17105.387097,17985.870968,13810.806452
std,0.297384,0.287008,0.295619,0.334207,12.082245,12.626478,11.245644,9.103822,9.103822,4.951376,24049.223531,24448.639346,21750.937461,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,44.000000,44.000000,0.000000
25%,0.338497,0.428638,0.408833,0.179300,8.000000,7.500000,3.000000,2.000000,2.000000,1.500000,764.500000,753.500000,375.500000,802.000000,802.000000,410.000000
50%,0.558096,0.599966,0.589826,0.472939,11.000000,10.000000,6.000000,6.000000,6.000000,4.000000,3234.000000,3160.000000,2309.000000,3847.000000,4138.000000,1944.000000
75%,0.834007,0.861489,0.875078,0.778451,25.000000,24.500000,13.000000,13.500000,13.500000,7.000000,29258.000000,29396.000000,27112.500000,26305.000000,27036.500000,23520.500000
max,0.947723,0.952594,0.955062,0.935512,51.000000,53.000000,52.000000,43.000000,43.000000,21.000000,89827.000000,89912.000000,80350.000000,102770.000000,105729.000000,92221.000000
